In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
import math
from collections import OrderedDict

import matplotlib as mpl
mpl.use('pdf')
import matplotlib.pyplot as plt

machine_names= ['silver1'] #'testbed-I_Tesla-K40', 
cool_names= ['testbed-II'] #'testbed-I', 
funcs=['Dgemm'] # 
dev_id = 0

def GigaVal_per_s(val, time):
	return val * 1e-9 / time

def dgemm_flops(M,N,K):
	return M*N*(2*K+1)

In /home/petyros/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/petyros/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/petyros/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/petyros/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/petyros/.local/lib/python3.6/site-packages/matplotlib/mpl-d

In [2]:
import plot_stuff as mplt 

font=8
plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=font-1)
plt.rc('ytick', labelsize=font-1)
plt.rc('axes', labelsize=font)


# width as measured in inkscape
width = 7.2
height = width / 1.618 * 3/5

import seaborn as sns
#sns.set(style="whitegrid", palette=mplt.cp2v1,font_scale=0.2, rc={"lines.linewidth": 1})


['9bd0be', '7fc0c0', '64b2c3', '49a3c5', '4386a0', '3e6a7b', '384e56', '9cce89', '7cbd78', '5cab67', '3c9956', '3a7f4d', '376544', '354c3b', 'ef8872', 'e86d5c', 'e15246', 'd93830', 'af3631', '853531', '5b3432']


In [3]:
import input_parsing as parse

exchept_T_list = [256,512,768,666]
#modelist_names = ['WERKHOVEN', 'WERKHOVEN_DATALOC', 'WERKHOVEN_LOOKUP_EXEC_TILES', 'COCOPELIA_DATALOC', 'COCOPELIA_BIDIRECTIONAL', 'COCOPELIA_REUSE', 'COCOPELIA_PIPELINE_EMULATE']
modelist_names = ['WERKHOVEN_DATALOC',  'COCOPELIA_BASELINE', 'COCOPELIA_DATALOC',  'COCOPELIA_BIDIRECTIONAL', 'COCOPELIA_REUSE'] #, 'WERKHOVEN_LOOKUP_EXEC_TILES'
model_cute_names = ['CSO-Model [11]', 'Baseline-Model (1)', 'Dataloc-Model (2)', 'BTS-Model (4)', 'DR-Model (5)'] #,'CSO[11]+Lookup'
marker_list = ['+', 'x', 's', 'd', 'o']
for TransA in ['N', 'T']:
    for TransB in ['N', 'T']:
        ctr = 0
        for machine in machine_names:
            functr = 0
            for func in funcs:
                print('Function: %s Machine : %s' % (func, machine))
                #fig, axs = plt.subplots(1,2, gridspec_kw={'width_ratios': [3, 1]})
                fig = plt.figure(constrained_layout=True)
                gs = fig.add_gridspec(4, 4)
                axs0 = fig.add_subplot(gs[0:, :3])
                axs1 = fig.add_subplot(gs[3, 3:])
                #fig.subplots_adjust(left=.07, bottom=.20*3/2, right=.99, top=.99)
                #validation_data = parse.read_validation_values("/home/users/panastas/PhD_stuff/CoCoPeLia-Framework/silver1_build/silver1-install/Library_scheduler/testLogs",func)
                validation_data = parse.read_validation_values("~/mount/PhD_stuff/CoCoPeLia-Framework/silver1_build/silver1-install/Library_scheduler/testLogs",func)
                dev_id_token = parse.tokenize_device(dev_id)
                validation_data = validation_data[validation_data['dev_ids_token']==dev_id_token]
                for col in ['Aloc','Bloc','Cloc']:
                    validation_data[col] = validation_data[col].apply(lambda x: parse.remote_f(x, dev_id))
                validation_data['cpu_ratio'] = validation_data['cpu_ratio'].apply(lambda x: int(x*100))
                validation_data = validation_data[(validation_data['TransA'] == TransA) & (validation_data['TransB'] == TransB)]
                #print(validation_data)
                total_set = pd.DataFrame()
                modelctr = 0
                rel_total_df = pd.DataFrame()
                rel_total_df_temp = pd.DataFrame()
                for model in modelist_names:

                    validation_pred = parse.read_validation_single_prediction("~/mount/PhD_stuff/CoCoPeLia-Framework/silver1_build/silver1-install/Autotuning_Runtime/testLogs/predictions",func,dev_id, model)
                    validation_pred['T'], validation_pred['cpu_ratio'], validation_pred['pred_t'] = parse.split_pred_struct(validation_pred['pred_struct'])
                    validation_pred['cpu_ratio'] = validation_pred['cpu_ratio'].apply(lambda x: int(x*100))
                    validation_pred = validation_pred[(validation_pred['TransA'] == TransA) & (validation_pred['TransB'] == TransB)]
                    #print(validation_pred)
                    validation_mixed = parse.create_validation_set(validation_data, validation_pred, func)
                    print(validation_mixed)
                    #Square set
                    locs= [[1,1,1], [1,1,0], [1,0,0], [1,0,1], [0,1,1], [0,1,0], [0,0,1]]
                    mid_sizes= [4096,8192,12288,16384]
                    ctrs_fat = [2]#[2,4,5,6,7,8]
                    ctrs_thin = [] #[5,7,10,13,16]
                    square_validation_set = parse.validation_set_split_BLAS3("Square",locs,mid_sizes,ctrs_fat,ctrs_thin,validation_mixed)
                    #parse.create_statistics(square_validation_set, 'CoCopeLia_t', 'CoCopelia', 'werkhoven')

                    #Fat&Thin set
                    locs= [[1,1,1]]
                    mid_sizes= [4096,8192,12288,16384]
                    ctrs_fat = [3,4,5]
                    ctrs_thin = [3,4,5]
                    fathin_validation_set = parse.validation_set_split_BLAS3("Fathin",locs,mid_sizes,ctrs_fat,ctrs_thin,validation_mixed)
                    #parse.create_statistics(fathin_validation_set, 'CoCopeLia_t', 'CoCopelia', 'werkhoven')

                    print('Model: %s' % (model))

                    x_axis = 'func'

                    united_set = pd.concat([square_validation_set, fathin_validation_set])
                    united_set = parse.cleanT(united_set,exchept_T_list)
                    #parse.create_statistics_single(united_set, 'CoCopeLia_t', 'pred_t')

                    perper_state = []
                    perper_model_perf = []
                    perper_static_sz = 2048#,4096
                    perper_static_perf = []
                    perper_best_perf = []

                    teams = united_set.groupby(['M', 'N' , 'K', 'Aloc', 'Bloc', 'Cloc'])
                    print("Validation sizes explored: %d" % len(teams))
                    for state, curr_set in teams:
                        perper_state.append(tuple(map(lambda x: int(int(x)/1024 + int(x)%2) ,state)))
                        #print("------------------------")
                        #print(curr_set.head(2), end="\n\n")
                        perper_model_perf.append(GigaVal_per_s(dgemm_flops(curr_set['M'].min(), curr_set['N'].min(),curr_set['K'].min()), 
                                                               float(curr_set.iloc[curr_set['pred_t'].argmin()]['CoCopeLia_avg_t'])))
                        #print ("Prediction Perf achieved : %.2lf Gflops/s" % perper_model_perf[-1])
                        perper_best_perf.append(GigaVal_per_s(dgemm_flops(curr_set['M'].min(), curr_set['N'].min(),curr_set['K'].min()), 
                                                              float(curr_set['CoCopeLia_avg_t'].min())))
                        if (curr_set[curr_set['T'] == perper_static_sz].empty == False):
                            print(curr_set[curr_set['T'] == perper_static_sz]['CoCopeLia_avg_t'])
                            perper_static_perf.append(float(GigaVal_per_s(dgemm_flops(curr_set['M'].min(), curr_set['N'].min(),curr_set['K'].min()), 
                                                                            float(curr_set[curr_set['T'] == perper_static_sz]['CoCopeLia_avg_t']))))
                        else:
                            perper_static_perf.append(float(GigaVal_per_s(dgemm_flops(curr_set['M'].min(), curr_set['N'].min(),curr_set['K'].min()), 
                                                                            float(curr_set[curr_set['T'] == curr_set['T'].max()]['CoCopeLia_avg_t']))))
                        #print ("Static T=%d Perf achieved : %.2lf Gflops/s" % (perper_static_sz, perper_static_perf[-1]))         

                    if rel_total_df.empty:
                        rel_perf_best = list(map(lambda x,y : float(x)/float(y), perper_best_perf, perper_static_perf)) 
                        rel_total_df['Perf'] = rel_perf_best
                        rel_total_df['Model'] = '$T_{opt}$'
                    rel_perf_list = list(map(lambda x,y : float(x)/float(y), perper_model_perf, perper_static_perf)) 
                    rel_total_df_temp['Perf'] = rel_perf_list
                    rel_total_df_temp['Model'] = model
                    rel_total_df = pd.concat([rel_total_df, rel_total_df_temp])

                    if total_set.empty:
                        total_set = united_set
                        #print(perper_state)
                        #print(perper_static_perf)
                        x =np.arange(len(perper_state))
                        axs0.grid(axis='y', linestyle='--', linewidth=0.5)
                        axs0.bar(x, perper_static_perf,color='lightgray')
                        axs0.plot(x, perper_best_perf, 'v', markersize=4 , label = "$T_{opt}$")
                        axs0.set_xticks(x)
                        axs0.set_xticklabels(perper_state, rotation = 90)
                        axs0.set_ylim(bottom=0.8*min(min(perper_model_perf), min(perper_static_perf)))
                        axs0.set_ylabel("Performance (Gflops/s)")
                        axs0.set_xlabel("( $\{M, N, K\}\cdot2^{10}, \{A, B, C\}loc=\{GPU:0,CPU:1\}$ )")
                    else:
                        total_set = pd.concat([total_set.copy(), united_set.copy()])
                    x = np.arange(len(perper_state))
                    axs0.plot(x, perper_model_perf, marker_list[modelctr], markersize=4, label = model_cute_names[modelctr])
                    modelctr +=1
                #print(rel_total_df)
                #print(total_set)
                #fig.savefig('./Plots/Validation_plot_violins_all_mine_reuse_Xaxis-%s.png' % (x_axis) )
                fig.legend(loc='lower right',ncol=1, fontsize=font-1)
                fig.set_size_inches(width, height)
                #fig.savefig('MakeTableIVGreatAgain_StaticT-%d_func-%s_%s.pdf' % (perper_static_sz, func, cool_names[ctr]) )
                #plt.clf()
                rel_total_df['func'] = func
                sns.boxplot(x='func',y='Perf',data=rel_total_df, hue ='Model', linewidth = 1, ax=axs1) #, bw=.05
                axs1.get_legend().remove()
                axs1.set_xticklabels([])
                axs1.set_xlabel('')
                axs1.set_ylabel('Relative Normalized Performance')
                axs1.set_xticks([])
                fig.savefig('BOX_MakeTableIVGreatAgain_StaticT-%d_func-%s_%s.pdf' % (perper_static_sz, func, cool_names[ctr]) )
                functr += 1
            ctr += 1

Function: Dgemm Machine : silver1
read_validation_values : Read 1716 values from "~/mount/PhD_stuff/CoCoPeLia-Framework/silver1_build/silver1-install/Library_scheduler/testLogs/CoCopeLiaDgemmRunner_predefined_vals.log"...
read_validation_values : Read 0 values from "~/mount/PhD_stuff/CoCoPeLia-Framework/silver1_build/silver1-install/Library_scheduler/testLogs/cuBLASXtDgemmRunner_predefined_vals.log"..
read_validation_single_prediction : Read 1792 predictions from "'~/mount/PhD_stuff/CoCoPeLia-Framework/silver1_build/silver1-install/Autotuning_Runtime/testLogs/predictions/CoCopeLiaLogPrediction-Dgemm_dev-0.log"...
read_validation_single_prediction : Kept 224 predictions for ModelName=WERKHOVEN_DATALOC from "'~/mount/PhD_stuff/CoCoPeLia-Framework/silver1_build/silver1-install/Autotuning_Runtime/testLogs/predictions/CoCopeLiaLogPrediction-Dgemm_dev-0.log"...
validation_set_split_BLAS3: 364 pairs in the clean validation set
validation_set_split_BLAS3: 22 pairs in the clean validation set
M

TypeError: cannot convert the series to <class 'float'>